In [1]:
import os


In [2]:
pwd

'c:\\Users\\Devendra\\Projects\\EmailSpamClassification\\research'

In [3]:
cd ..

c:\Users\Devendra\Projects\EmailSpamClassification


In [4]:
pwd

'c:\\Users\\Devendra\\Projects\\EmailSpamClassification'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    target_column: str

In [6]:
from EmailSpamClassification.constants import *
from EmailSpamClassification.utils.utils import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            target_column = schema.name
        )

        return data_transformation_config


In [8]:
import os
from EmailSpamClassification import logger
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import re
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
from sklearn.feature_extraction.text import CountVectorizer


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Devendra\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    
    def get_cleaned_data(self,dataframe):
        try:

            corpus = []
            for i in range(0, len(dataframe)):
                review = re.sub('[^a-zA-Z]', ' ', dataframe['text'][i])
                review = review.lower()
                review = review.split()
                review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
                review = ' '.join(review)

                corpus.append(review)
            return corpus
        
        except Exception as e:
            raise e


    def train_test_spliting(self):

        data = pd.read_csv(self.config.data_path)
        print(data.head())

        corpus = self.get_cleaned_data(data)
        ##print(corpus)
        print(self.config.target_column)
        target_feature = data[[self.config.target_column]]
        #print(target_feature)

        # Split the data into training and test sets. (0.75, 0.25) split.
        X_train, X_test, y_train, y_test = train_test_split(corpus,target_feature,test_size=0.2, random_state=42)

        cv = CountVectorizer(max_features=2500)
        # Fit and transform with CountVectorizer
        X_train = cv.fit_transform(X_train).toarray()
        X_test = cv.transform(X_test).toarray()

        # Get feature names from CountVectorizer
        cols = list(cv.get_feature_names_out())

        columns_name = cols + [self.config.target_column]
        print(columns_name)

        print(X_train.shape)
        print(X_test.shape)
        print(y_train.shape)
        print(y_test.shape)
     

        # Create DataFrames
        train_data = np.column_stack((X_train, y_train))
        test_data = np.column_stack((X_test, y_test))

        train_data = pd.DataFrame(train_data, columns= columns_name)
        test_data = pd.DataFrame(test_data, columns= columns_name)


        # Save DataFrames to CSV
        train_data.to_csv(os.path.join(self.config.root_dir, "train.csv"), index=False)
        test_data.to_csv(os.path.join(self.config.root_dir, "test.csv"), index=False)

        logger.info("Splited data into training and test sets")
        logger.info(train_data.shape)
        logger.info(test_data.shape)

        print(train_data.shape)
        print(test_data.shape)
        

In [12]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_spliting()
except Exception as e:
    raise e

[2024-01-28 17:04:54,024: INFO: utils: yaml file: config\config.yaml loaded successfully]
[2024-01-28 17:04:54,032: INFO: utils: yaml file: params.yaml loaded successfully]
[2024-01-28 17:04:54,036: INFO: utils: yaml file: schema.yaml loaded successfully]
[2024-01-28 17:04:54,036: INFO: utils: created directory at: artifacts]
[2024-01-28 17:04:54,036: INFO: utils: created directory at: artifacts/data_transformation]
                                                text  spam
0  Subject: naturally irresistible your corporate...     1
1  Subject: the stock trading gunslinger  fanny i...     1
2  Subject: unbelievable new homes made easy  im ...     1
3  Subject: 4 color printing special  request add...     1
4  Subject: do not have money , get software cds ...     1
spam
['aa', 'ab', 'abil', 'abl', 'absolut', 'abstract', 'ac', 'academ', 'acadia', 'accept', 'access', 'accommod', 'accomplish', 'accord', 'account', 'accur', 'accuraci', 'achiev', 'acquir', 'acquisit', 'acrobat', 'across', 'ac